In [1]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings


MODEL = "llama2"

model=Ollama(model=MODEL)
embeddings=OllamaEmbeddings()
print(model.invoke("tell me a joke"))



Sure, here's one:

Why don't scientists trust atoms?
Because they make up everything!

I hope you found that amusing! Do you have any specific topic or theme in mind for a joke, or would you like me to come up with something else?


In [2]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()


loader = PyPDFLoader("awsgsg-intro (2).pdf")
pages = loader.load_and_split()
print(pages)

[Document(page_content='Getting Star ted with A WS', metadata={'source': 'awsgsg-intro (2).pdf', 'page': 0}), Document(page_content="Getting Star ted with A WS\nCopyright © 2014 Amaz on Web Ser vices , Inc.  and/or its affiliates . All rights reser ved.\nThe f ollowing are tr ademar ks of Amaz on Web Ser vices , Inc.:  Amaz on, Amaz on Web Ser vices Design, A WS, Amaz on CloudF ront,\nCloudfront, Amaz on De vPay, DynamoDB , ElastiCache , Amaz on EC2, Amaz on Elastic Compute Cloud, Amaz on Glacier , Kindle , Kindle\nFire, AWS Mar ketplace Design, Mechanical Turk, Amaz on Redshift, Amaz on Route 53, Amaz on S3, Amaz on VPC. In addition,\nAmaz on.com g raphics , logos , page headers , button icons , scripts, and ser vice names are tr ademar ks, or tr ade dress of Amaz on in\nthe U .S. and/or other countr ies. Amaz on's tr ademar ks and tr ade dress ma y not be used in connection with an y product or ser vice that\nis not Amaz on's, in an y manner that is lik ely to cause confusion among c

In [3]:
from langchain.prompts import PromptTemplate
template = """
Answer the question if you cant
answer the question , reply "i dont know".

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
prompt.format(context='here is some context',question="here is some question" )

'\nAnswer the question if you cant\nanswer the question , reply "i dont know".\n\nContext: here is some context\n\nQuestion: here is some question\n\n'

In [4]:
chain = prompt|model|parser

In [5]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [6]:
chain.invoke(
    {
        "context" : "The name was om",
        "question" : "what is my name?"
    }
)

"I'm just an AI, I don't have access to personal information or the ability to know the names of individuals. Therefore, I cannot answer your question about your name. I'm just an AI and do not have the capability to access or know personal information. If you would like to share your name with me, I would be happy to learn it!"

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

loader = PyPDFLoader("awsgsg-intro (2).pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embedding_function)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs)

[Document(page_content='Getting Star ted with A WS', metadata={'page': 0, 'source': 'awsgsg-intro (2).pdf'}), Document(page_content='Getting Star ted with A WS', metadata={'page': 0, 'source': 'awsgsg-intro (2).pdf'}), Document(page_content='Tutorial Amazon Web Ser vice\nUpdating a Stac k AWS CloudFormation\nGetting Star ted with CloudW atch Amazon CloudW atch\nGetting Star ted with A uto Scaling Auto Scaling\n26Getting Star ted with A WS\nGet Star ted', metadata={'page': 28, 'source': 'awsgsg-intro (2).pdf'}), Document(page_content='Tutorial Amazon Web Ser vice\nUpdating a Stac k AWS CloudFormation\nGetting Star ted with CloudW atch Amazon CloudW atch\nGetting Star ted with A uto Scaling Auto Scaling\n26Getting Star ted with A WS\nGet Star ted', metadata={'page': 28, 'source': 'awsgsg-intro (2).pdf'})]


In [9]:
retriver=db.as_retriever()
retriver.invoke("machine learning")

[Document(page_content='In each categor y, there are one or more ser vices . For example , AWS off ers fiv e database ser vices , each\none optimiz ed for a cer tain type of use .With so man y offerings, you can design an A WS solution that is\ntailored to y our needs .\n2Getting Star ted with A WS\nWhat Can I Do with A WS?', metadata={'page': 4, 'source': 'awsgsg-intro (2).pdf'}),
 Document(page_content='In each categor y, there are one or more ser vices . For example , AWS off ers fiv e database ser vices , each\none optimiz ed for a cer tain type of use .With so man y offerings, you can design an A WS solution that is\ntailored to y our needs .\n2Getting Star ted with A WS\nWhat Can I Do with A WS?', metadata={'page': 4, 'source': 'awsgsg-intro (2).pdf'}),
 Document(page_content="Getting Star ted with A WS\nCopyright © 2014 Amaz on Web Ser vices , Inc.  and/or its affiliates . All rights reser ved.\nThe f ollowing are tr ademar ks of Amaz on Web Ser vices , Inc.:  Amaz on, Amaz on W

In [12]:
from operator import itemgetter
chain =(
{
    "context":itemgetter("question") | retriver,
    "question":itemgetter("question")

}
|prompt
|model
|parser
)
chain.invoke({"question":"what is machine learning"})

'I can answer the question or reply "I don\'t know" depending on my understanding of the context and the information provided. Please provide more details or context about the question you want me to answer, and I will do my best to assist you.'

In [13]:
questions = [
    "what is aws ?",
    "how may hours is required to study"
]

for question in questions:
    print(f"Question:{question}")
    print(f"Answer:{chain.invoke({'question':question})}")
    print()

Question:what is aws ?
Answer:I don't know the answer to that question. The document you provided doesn't mention anything about AWS. Could you please provide more context or clarify which AWS service you are asking about?

Question:how may hours is required to study
Answer:The amount of time required to study AWS and its services will depend on various factors, such as your level of expertise, the depth of knowledge you want to acquire, and the pace at which you learn. Here are some general guidelines:

1. Introduction to AWS: 2-4 hours - This will give you a basic understanding of what AWS is, its services, and how it works.
2. Understanding AWS services: 8-10 hours - Once you have a good understanding of AWS, you can start exploring the various services offered by AWS. Each service has its own documentation, tutorials, and examples that can help you learn how to use them effectively.
3. Hands-on experience with AWS: 10-20 hours - After familiarizing yourself with the services, it's 